<a href="https://colab.research.google.com/github/fermuba/ML_OPS_MOVIES/blob/main/ETL/DS_credits_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Limpieza y extracion de datos de Directores

In [ ]:
# Acceso a los archivos de Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Carga de Librerias de uso
import pandas as pd
import numpy as np
import json
import ast # Abstract Syntax Trees

# configuro semilla de random
np.random.seed(0)

In [ ]:
# Insgesta del archivos movie_dataset.csv
credits_crew = pd.read_csv('/content/drive/MyDrive/Colab/HENRY/PI-1/Data_font/credits_crew.csv')

In [ ]:
# Informacion del dataframe
credits_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  45476 non-null  int64 
 1   id          45476 non-null  int64 
 2   crew        45476 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.0+ MB


In [ ]:
# Encabezado del Dataframe
credits_crew.head()

,Unnamed: 0,id,crew
0,0,862,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,1,8844,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,2,15602,"[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,3,31357,"[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,4,11862,"[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [ ]:
# Convertir la cadena str en una cadena de datos python
credits_crew['crew'] = credits_crew['crew'].apply(ast.literal_eval)

In [ ]:
# Expando los elementos json uno por columna
columna_expandida = pd.json_normalize(credits_crew['crew'])
credits_crew = pd.concat([credits_crew, columna_expandida], axis= 1)

In [ ]:
# Remplazo nombre de columna id
credits_crew.rename(columns= {'id' : 'id_movie'}, inplace= True)

In [ ]:
# Nuevo tamaño
credits_crew.shape

(45476, 438)

In [ ]:
# Listado del nombre de columnas
credits_crew.columns

Index(['Unnamed: 0',   'id_movie',       'crew',            0,            1,
                  2,            3,            4,            5,            6,
       ...
                425,          426,          427,          428,          429,
                430,          431,          432,          433,          434],
      dtype='object', length=438)

In [ ]:
# Info nueva
credits_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Columns: 438 entries, Unnamed: 0 to 434
dtypes: int64(2), object(436)
memory usage: 152.0+ MB


In [ ]:
# Realizo una nueva expancion a los datos obtenidos
for columna in range(0, 435):
  columna_expandida = pd.json_normalize(credits_crew[columna])
  credits_crew = pd.concat([credits_crew, columna_expandida], axis= 1)

In [ ]:
# Eliminacion de columnas que no vamos a usar
columnas_eliminar = np.arange(0, 435)
credits_crew.drop(columns= columnas_eliminar, inplace= True)
credits_crew.drop(columns= 'crew', inplace= True)

In [ ]:
# Nuevo tamaño
credits_crew.shape

(45476, 3047)

In [ ]:
# Eliminacion de columnas que no vamos a usar
columnas_eliminar = ['credit_id', 'id', 'gender', 'profile_path']
credits_crew.drop(columns= columnas_eliminar, inplace= True)

In [ ]:
# Nuevo tamaño
credits_crew.shape

(45476, 1307)

In [ ]:
# Se aplica un filtro por departamento direccion y cargo director, las coincidencias se almacenan en un df de 2 columnas
# Creacion de un Df vacio para cargar los datos obtenidos
credits_crew_new = pd.DataFrame()

# Se recoren las columnas, la secuencia se repite porque son datos json con la misma estructura por eso con solo tener
# un solo indice se pueden ubicar los anteriores
for column in range(4 , 1307):
  filtro = (credits_crew.iloc[:, (column - 2)] == 'Directing') & (credits_crew.iloc[:, (column - 1)] == 'Director')
  credits_crew_new = pd.concat([credits_crew_new, credits_crew[filtro].iloc[:, [1, column]]], axis= 0)

In [ ]:
# Eliminacion de columnas innecesarias
credits_crew_new.drop(columns= ['department', 'job'], inplace= True)

In [ ]:
# Se crea un nuevo indice para el df
credits_crew_new.sort_index(inplace= True)

In [ ]:
# Nuevo Df
credits_crew_new.head()

,id_movie,name
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [ ]:
# Eliminacion de duplicados, se comparan por par de datos (id_movie, name)
indices_duplicados = credits_crew_new[credits_crew_new.duplicated(subset= ['id_movie', 'name'], keep= 'first')].index
credits_crew_new.drop(index= indices_duplicados, inplace= True)

In [ ]:
# Se guardan en archivo los datos obtenidos
credits_crew_new.to_csv('/content/drive/MyDrive/Colab/HENRY/PI-1/Data_font/credits_crew_clean.csv')